In [78]:
import yfinance as yf
import datetime
import pandas as pd
import plotly.graph_objects as go
import math
import plotly.io as pio
import numpy as np
import pickle
import os
TODAY = datetime.datetime.today()
# TODAY = (TODAY-datetime.timedelta(days=2))
print(TODAY)

2021-12-04 19:29:24.403751


In [79]:
def get_nifty50_stocks():
    #PROCESS AND LOAD NIFTY 50 DATA
    nif50 = pd.read_html('https://www.moneyseth.com/blogs/Nifty-50-Stock-List-and-its-Weightage-in-Index')[1]
    new_header = nif50.iloc[0] #grab the first row for the header
    nif50 = nif50[1:] #take the data less the header row
    nif50.columns = new_header #set the header row as the df header
    nif50.drop(columns='Sl No', axis=1, inplace=True)
    return nif50

def get_data_date(SYM, START=None, END=None):
    if SYM[0]=='^' or '=' in SYM:
        NAME = f'{SYM}'
    else:
        NAME = f'{SYM}.NS'
    
    df = yf.download(NAME,
                 start=START,
                 end=END,
                 progress=False)
    df.reset_index(inplace=True)

    #RESISTANCE/SUPPORT FINDING
    df['Topline'] = df.apply(lambda x : x.Open if x.Open>x.Close else x.Close, axis=1)
    df['Bottomline'] = df.apply(lambda x : x.Open if x.Open<x.Close else x.Close, axis=1)

    #Create 200 period Moving Average
    df['Moving_Average_200'] = df.Close.rolling(window=200).mean().shift(1)
    df['Moving_Average_50'] = df.Close.rolling(window=50).mean().shift(1)
    df['Moving_Average_20'] = df.Close.rolling(window=20).mean().shift(1)
    
    df['ATR1'] = df.High-df.Low
    df['ATR2'] = abs(df.High-df.Close.shift(-1))
    df['ATR3'] = abs(df.Low-df.Close.shift(-1))
    
    
    df['ATR'] = df[['ATR1','ATR2', 'ATR3']].max(axis=1) # USE to get level area range
    df['200_Avg_ATR'] = df.ATR.rolling(window=200).mean()
    df.drop(columns=['ATR1','ATR2', 'ATR3'], inplace=True)
    
    return df

In [80]:
def detect_doji(o,h,l,c):
    body = abs(o-c)
    rng  = abs(h-l)
    
    if body/rng <=0.25:
        return True
    return False

def detect_bullish_engulfing(po,pc,o,c):
    if po>pc and o<c and c>po and o < pc:
        return True
    return False

def detect_bearish_engulfing(po,pc,o,c):
    if po<pc and o>c and c<po:
        return True
    return False

In [93]:
def get_rsi(df, cols=None, n=14):
    df['change'] = (df.Close - df.Close.shift(1))/df.Close.shift(1)
    df['up_change'] = df.change.apply(lambda x: x if x>0 else 0)
    df['up_mean_change'] = df.up_change.rolling(n).mean()
    df['down_change'] = df.change.apply(lambda x: x if x<0 else 0)
    df['down_mean_change'] = df.down_change.rolling(n).mean()
    df[f'rsi_{n}'] = 100-(100/(1+(df.up_mean_change/(-df.down_mean_change))))
    if cols is None:
        df.drop(columns=['up_change','up_mean_change','down_change','down_mean_change'], inplace=True)
        return df
    return df[cols]

def get_resistance_level(df):
    df['Topline-1'] = df.Topline - df.Topline.shift(-1)
    df['Topline-2'] = df.Topline - df.Topline.shift(-2)
    df['Topline-3'] = df.Topline - df.Topline.shift(-3)
    df['Topline-4'] = df.Topline - df.Topline.shift(-4)
    df['Topline+1'] = df.Topline - df.Topline.shift(1)
    df['Topline+2'] = df.Topline - df.Topline.shift(2)
    df['Topline+3'] = df.Topline - df.Topline.shift(3)
    df['Topline+4'] = df.Topline - df.Topline.shift(4)
    
    df['is_Resistance'] = (df['Topline-1']>=0)&(df['Topline-2']>=0)&(df['Topline+1']>=0)&(df['Topline+2']>=0)
    df['is_Resistance_strong'] = (df['is_Resistance'])&(df['Topline-3']>=0)&(df['Topline-4']>=0)&(df['Topline+3']>=0)&(df['Topline+4']>=0)
    df.drop(columns= ['Topline-1','Topline-2','Topline-3','Topline-4','Topline+1','Topline+2','Topline+3','Topline+4'], inplace=True)
    return df

def get_support_level(df):
    df['Bottomline-1'] = df.Bottomline - df.Bottomline.shift(-1)
    df['Bottomline-2'] = df.Bottomline - df.Bottomline.shift(-2)
    df['Bottomline-3'] = df.Bottomline - df.Bottomline.shift(-3)
    df['Bottomline-4'] = df.Bottomline - df.Bottomline.shift(-4)
    df['Bottomline+1'] = df.Bottomline - df.Bottomline.shift(1)
    df['Bottomline+2'] = df.Bottomline - df.Bottomline.shift(2)
    df['Bottomline+3'] = df.Bottomline - df.Bottomline.shift(3)
    df['Bottomline+4'] = df.Bottomline - df.Bottomline.shift(4)
    
    df['is_Support'] = (df['Bottomline-1']<=0)&(df['Bottomline-2']<=0)&(df['Bottomline+1']<=0)&(df['Bottomline+2']<=0)
    df['is_Support_strong'] = (df['is_Support'])&(df['Bottomline-3']<=0)&(df['Bottomline-4']<=0)&(df['Bottomline+3']<=0)&(df['Bottomline+4']<=0)
    df.drop(columns= ['Bottomline-1','Bottomline-2','Bottomline-3','Bottomline-4','Bottomline+1','Bottomline+2','Bottomline+3','Bottomline+4'], inplace=True)
    return df

#THIS NEEDS TO BE REFINED AND TESTED
def get_trend(df): 
    def get_strength(cng):
        cng = abs(cng)
        if cng < 0.12 :
            return 'Weak '
        elif cng <= 0.3:
            return 'Healthy '
        elif cng > 0.3:
            return 'Strong '
        else:
            return '-'
    df['cng'] = (df.Close - df.Close.shift(250))/df.Close.shift(250)
    df['Trend'] = df['cng'].apply(lambda x : get_strength(x)+('Up' if x>0 else ('Down' if x<0 else '-')))
    df.drop(columns=['cng'], inplace=True)
    return df

def which_level_support_probable(df):
    cp = df.iloc[-1].Close
    try:
        support_1 = df[(df.is_Support)&(df.Bottomline<cp)].iloc[-1]['Bottomline']
    except:
        return ('Not Sure', None)
    try:
        support_2 = df[(df.is_Support)&(df.Bottomline<cp)].iloc[-2]['Bottomline']
    except:
        return ('Not Sure', support_1)
    
    
    if abs(support_1-support_2) <= df[df.is_Support].iloc[-1]['ATR']:
        return ('Support_Level', support_1)
    if (abs(support_1-df[df.is_Support].iloc[-1]['Moving_Average_20']) <= df[df.is_Support].iloc[-1]['ATR'] and
       abs(support_2-df[df.is_Support].iloc[-2]['Moving_Average_20']) <= df[df.is_Support].iloc[-2]['ATR'] and
       df.iloc[-1]['Moving_Average_20']<cp):
        return ('Moving_Average_20', df.iloc[-1]['Moving_Average_20'])
    
    if (abs(support_1-df[df.is_Support].iloc[-1]['Moving_Average_50']) <= df[df.is_Support].iloc[-1]['ATR'] and
       abs(support_2-df[df.is_Support].iloc[-2]['Moving_Average_50']) <= df[df.is_Support].iloc[-2]['ATR'] and
       df.iloc[-1]['Moving_Average_50']<cp):
        return ('Moving_Average_50', df.iloc[-1]['Moving_Average_50'])
    
    if (abs(support_1-df[df.is_Support].iloc[-1]['Moving_Average_200']) <= df[df.is_Support].iloc[-1]['ATR'] and
       abs(support_2-df[df.is_Support].iloc[-2]['Moving_Average_200']) <= df[df.is_Support].iloc[-2]['ATR'] and
       df.iloc[-1]['Moving_Average_200']<cp):
        return ('Moving_Average_200', df.iloc[-1]['Moving_Average_200'])
    
    return ('Not Sure', support_1)

def which_level_resistance_probable(df):
    cp = df.iloc[-1].Close
    try:
        resistance_1 = df[(df.is_Resistance)&(cp<df.Topline)].iloc[-1]['Topline']
    except:
        return ('Not Sure', None)
    try:
        resistance_2 = df[(df.is_Resistance)&(cp<df.Topline)].iloc[-2]['Topline']
    except:
        return ('Not Sure', resistance_1)
    
    
    if abs(resistance_1-resistance_2) <= df[df.is_Resistance].iloc[-1]['ATR']:
        return ('Resistance_Level', resistance_1)
    if (abs(resistance_1-df[df.is_Resistance].iloc[-1]['Moving_Average_20']) <= df[df.is_Resistance].iloc[-1]['ATR'] and
       abs(resistance_2-df[df.is_Resistance].iloc[-2]['Moving_Average_20']) <= df[df.is_Resistance].iloc[-2]['ATR'] and 
       df.iloc[-1]['Moving_Average_20']>cp):
        return ('Moving_Average_20', df.iloc[-1]['Moving_Average_20'])
    
    if (abs(resistance_1-df[df.is_Resistance].iloc[-1]['Moving_Average_50']) <= df[df.is_Resistance].iloc[-1]['ATR'] and
       abs(resistance_2-df[df.is_Resistance].iloc[-2]['Moving_Average_50']) <= df[df.is_Resistance].iloc[-2]['ATR'] and
       df.iloc[-1]['Moving_Average_50']>cp):
        return ('Moving_Average_50', df.iloc[-1]['Moving_Average_50'])
    
    if (abs(resistance_1-df[df.is_Resistance].iloc[-1]['Moving_Average_200']) <= df[df.is_Resistance].iloc[-1]['ATR'] and
       abs(resistance_2-df[df.is_Resistance].iloc[-2]['Moving_Average_200']) <= df[df.is_Resistance].iloc[-2]['ATR'] and
       df.iloc[-1]['Moving_Average_200']>cp):
        return ('Moving_Average_200', df.iloc[-1]['Moving_Average_200'])
    
    return ('Not Sure', resistance_1)


In [103]:
def detect_area_of_value(cp, ATR, sup_level, res_level, trend, sup_type='Not Sure', res_type='Not Sure'):
    if 'Up' in trend:
#         if sup_type=='Not Sure':
#             return False
        try:
            sr_range = abs(sup_level-res_level)
        except:
            sr_range = 0
        if ((sup_level+ATR) >= cp >= (sup_level-ATR)) and cp<=(sup_level+sr_range/2):
            return True
        
    if 'Down' in trend:
        if res_type=='Not Sure':
            return False
        try:
            sr_range = abs(sup_level-res_level)
        except:
            sr_range = 0
        if ((res_level-ATR) <= cp >= (res_level+ATR)) and cp>=(res_level-sr_range/2):
            return True
    return False


def detect_trigger(df):
    if 'Up' in df.iloc[-1].Trend:
        if detect_doji(df.iloc[-1].Open, df.iloc[-1].High, df.iloc[-1].Low, df.iloc[-1].Close):
            if min(df.iloc[-1].Open, df.iloc[-1].Close)>=((df.iloc[-1].High+df.iloc[-1].Low)/2):
                return (True, 'Up_Doji')
        if detect_bullish_engulfing(df.iloc[-2].Open, df.iloc[-2].Close, df.iloc[-1].Open, df.iloc[-1].Close):
            return (True, 'Bullish_Engulfing')
        if (df.iloc[-2].rsi_14 < 30) and (df.iloc[-2].rsi_14 < df.iloc[-1].rsi_14) and not detect_doji(df.iloc[-1].Open, df.iloc[-1].High, df.iloc[-1].Low, df.iloc[-1].Close):
            return (True, 'RSI')
        
    if 'Down' in df.iloc[-1].Trend:
        if detect_doji(df.iloc[-1].Open, df.iloc[-1].High, df.iloc[-1].Low, df.iloc[-1].Close):
            if max(df.iloc[-1].Open, df.iloc[-1].Close)<=((df.iloc[-1].High+df.iloc[-1].Low)/3):
                return (True, 'Down_Doji')
        if detect_bearish_engulfing(df.iloc[-2].Open, df.iloc[-2].Close, df.iloc[-1].Open, df.iloc[-1].Close):
            return (True, 'Bearish_Engulfing')
        if (df.iloc[-2].rsi_14 > 80) and (df.iloc[-2].rsi_14 > df.iloc[-1].rsi_14) and not detect_doji(df.iloc[-1].Open, df.iloc[-1].High, df.iloc[-1].Low, df.iloc[-1].Close):
            return (True, 'RSI')
        
    return (False, None)

In [104]:
def process_df(df):
    df = get_resistance_level(df)
    df = get_support_level(df)
    df = get_trend(df)
    df = get_rsi(df)
    cp = df.iloc[-1].Close
    ATR = df.iloc[-1].ATR
    trend = df.iloc[-1].Trend
    MA20 = df.iloc[-1].Moving_Average_20
    MA50 = df.iloc[-1].Moving_Average_50
    MA200 = df.iloc[-1].Moving_Average_200
    sup_type, sup_level = which_level_support_probable(df)
    res_type, res_level = which_level_resistance_probable(df)
    is_in_area_of_value = detect_area_of_value(cp, ATR, sup_level, res_level, trend, sup_type, res_type)
    is_trigger, trigger_type = detect_trigger(df)
    
    return {'Close' : cp, 'ATR' : ATR, 'Trend' : trend, 'Support_Type' : sup_type, 'Support_Level' : sup_level,
            'Resistance_Type' : res_type, 'Resistance_Level' : res_level, 'In_Area_of_Value' : is_in_area_of_value,
            'Is_Trigger' : is_trigger, 'Trigger_Type' : trigger_type, 'Moving_Average_20':MA20, 'Moving_Average_50':MA50,
            'Moving_Average_200':MA200}

In [105]:
def get_for_bt(d,t):
    df = get_data_date(t,START=datetime.datetime.strptime(d,'%Y-%m-%d')-datetime.timedelta(days=450),END=datetime.datetime.strptime(d,'%Y-%m-%d')+datetime.timedelta(days=1)) #REMOVE TODAY.date() as END
    row = process_df(df)
    row['NAME'] = t
    row['Date'] = df.iloc[-1].Date
    row['Action'] = ('Buy' if ('Up' in row['Trend'] and row['In_Area_of_Value'] and row['Is_Trigger']) else(
                        'Sell' if ('Down' in row['Trend'] and row['In_Area_of_Value'] and row['Is_Trigger']) else None))
    row['Target'] = (row['Resistance_Level' if row['Resistance_Type']=='Not Sure' else row['Resistance_Type']]-0.2*row['ATR'] if row['Action']=='Buy' else(
                     row['Support_Level' if row['Support_Type']=='Not Sure' else row['Support_Type']]+0.2*row['ATR'] if row['Action']=='Sell' else None))
    
    row['Stoploss'] = (row['Resistance_Level' if row['Resistance_Type']=='Not Sure' else row['Resistance_Type']]+row['ATR'] if row['Action']=='Sell' else(
                         row['Support_Level' if row['Support_Type']=='Not Sure' else row['Support_Type']]-row['ATR'] if row['Action']=='Buy' else None))
    try:
        row['Expected_Within'] = int((abs(row['Close']-row['Target'])/((df.iloc[-1].ATR+df.iloc[-2].ATR+df.iloc[-3].ATR)/3))+1)
    except:
        row['Expected_Within'] = None
    try:  
        row['Risk_to_Reward'] = round((row['Close']-row['Stoploss'])/(row['Target']-row['Close']), 2)
    except:
        row['Risk_to_Reward'] = None
        
    return pd.DataFrame([row])

In [91]:
nif50 = get_nifty50_stocks()['NSE Symbol'].to_list()
nif50.extend(['^NSEI',"INR=X",'MCDOWELL-N'])

final = []
c = 0
t = len(nif50)
for eq in nif50:
    c += 1
    print(f'{c} out of {t} | Going for {eq}               ', end='\r')
    df = get_data_date(eq,str((TODAY-datetime.timedelta(days=450)).date()),str((TODAY+datetime.timedelta(days=1)).date())) #REMOVE TODAY.date() as END
    row = process_df(df)
    row['NAME'] = eq
    row['Date'] = df.iloc[-1].Date
    row['Action'] = ('Buy' if ('Up' in row['Trend'] and row['In_Area_of_Value'] and row['Is_Trigger']) else(
                        'Sell' if ('Down' in row['Trend'] and row['In_Area_of_Value'] and row['Is_Trigger']) else None))
    row['Target'] = (row['Resistance_Level' if row['Resistance_Type']=='Not Sure' else row['Resistance_Type']]-0.2*row['ATR'] if row['Action']=='Buy' else(
                     row['Support_Level' if row['Support_Type']=='Not Sure' else row['Support_Type']]+0.2*row['ATR'] if row['Action']=='Sell' else None))
    
    row['Stoploss'] = (row['Resistance_Level' if row['Resistance_Type']=='Not Sure' else row['Resistance_Type']]+row['ATR'] if row['Action']=='Sell' else(
                         row['Support_Level' if row['Support_Type']=='Not Sure' else row['Support_Type']]-row['ATR'] if row['Action']=='Buy' else None))
    try:
        row['Expected_Within'] = int((abs(row['Close']-row['Target'])/((df.iloc[-1].ATR+df.iloc[-2].ATR+df.iloc[-3].ATR)/3))+1)
    except:
        row['Expected_Within'] = None
    try:  
        row['Risk_to_Reward'] = round((row['Close']-row['Stoploss'])/(row['Target']-row['Close']), 2)
    except:
        row['Risk_to_Reward'] = None
    final.append(row)
    
df = pd.DataFrame(final)

          Date         Open         High          Low        Close  \
0   2020-09-10  2183.100098  2344.949951  2175.350098  2314.000000   
1   2020-09-11  2312.000000  2337.850098  2287.100098  2319.750000   
2   2020-09-14  2325.000000  2360.000000  2282.000000  2302.550049   
3   2020-09-15  2311.949951  2325.750000  2288.149902  2318.850098   
4   2020-09-16  2320.000000  2369.350098  2310.550049  2324.550049   
..         ...          ...          ...          ...          ...   
301 2021-11-29  2439.100098  2500.000000  2399.100098  2441.500000   
302 2021-11-30  2468.000000  2475.899902  2388.850098  2405.399902   
303 2021-12-01  2433.000000  2474.000000  2425.100098  2467.000000   
304 2021-12-02  2469.699951  2496.199951  2461.600098  2482.850098   
305 2021-12-03  2498.399902  2498.500000  2400.000000  2408.250000   

       Adj Close    Volume      Topline   Bottomline  Moving_Average_200  \
0    2306.579834  64750460  2314.000000  2183.100098                 NaN   
1    23

          Date        Open        High         Low       Close   Adj Close  \
0   2020-09-10  371.700012  375.000000  365.450012  370.700012  369.617767   
1   2020-09-11  373.000000  373.649994  365.100006  370.500000  369.418335   
2   2020-09-14  373.500000  373.850006  359.350006  363.700012  362.638184   
3   2020-09-15  364.049988  372.250000  363.700012  371.549988  370.465240   
4   2020-09-16  371.350006  376.700012  365.000000  374.700012  373.606079   
..         ...         ...         ...         ...         ...         ...   
301 2021-11-29  726.750000  727.750000  710.500000  718.400024  718.400024   
302 2021-11-30  716.750000  732.299988  711.099976  714.349976  714.349976   
303 2021-12-01  716.400024  732.750000  715.000000  727.700012  727.700012   
304 2021-12-02  724.599976  727.099976  715.200012  722.400024  722.400024   
305 2021-12-03  720.500000  729.500000  715.099976  716.299988  716.299988   

       Volume     Topline  Bottomline  Moving_Average_200  Movi

          Date         Open         High          Low        Close  \
0   2020-09-10  1330.000000  1343.050049  1315.099976  1319.500000   
1   2020-09-11  1317.000000  1336.199951  1311.000000  1330.349976   
2   2020-09-14  1337.000000  1347.949951  1300.000000  1304.699951   
3   2020-09-15  1308.099976  1328.849976  1289.000000  1323.949951   
4   2020-09-16  1324.000000  1332.000000  1300.550049  1325.400024   
..         ...          ...          ...          ...          ...   
301 2021-11-29  1993.000000  2045.000000  1937.199951  2019.599976   
302 2021-11-30  2023.000000  2039.599976  1946.900024  1961.900024   
303 2021-12-01  1979.000000  1985.900024  1940.000000  1953.349976   
304 2021-12-02  1947.550049  1980.949951  1945.599976  1964.250000   
305 2021-12-03  1966.699951  1989.900024  1905.199951  1914.199951   

       Adj Close   Volume      Topline   Bottomline  Moving_Average_200  \
0    1318.845093  3289948  1330.000000  1319.500000                 NaN   
1    1329

          Date        Open        High         Low       Close   Adj Close  \
0   2020-09-10  184.100006  185.699997  183.199997  184.899994  175.731583   
1   2020-09-11  184.899994  186.500000  182.750000  183.949997  174.828674   
2   2020-09-14  185.800003  185.850006  183.000000  183.350006  174.258438   
3   2020-09-15  184.000000  184.199997  180.800003  181.800003  172.785309   
4   2020-09-16  181.850006  183.500000  180.050003  180.649994  171.692307   
..         ...         ...         ...         ...         ...         ...   
301 2021-11-29  222.000000  223.649994  220.199997  221.500000  221.500000   
302 2021-11-30  222.000000  225.199997  220.000000  221.149994  221.149994   
303 2021-12-01  222.000000  223.300003  220.600006  221.899994  221.899994   
304 2021-12-02  221.899994  226.149994  221.300003  225.449997  225.449997   
305 2021-12-03  225.000000  226.199997  221.350006  221.649994  221.649994   

       Volume     Topline  Bottomline  Moving_Average_200  Movi

          Date        Open        High         Low       Close   Adj Close  \
0   2020-09-10  508.000000  508.000000  491.200012  497.350006  497.350006   
1   2020-09-11  499.299988  502.450012  488.549988  491.649994  491.649994   
2   2020-09-14  496.000000  497.250000  468.299988  474.100006  474.100006   
3   2020-09-15  475.149994  487.649994  470.049988  485.549988  485.549988   
4   2020-09-16  487.000000  490.700012  480.549988  481.750000  481.750000   
..         ...         ...         ...         ...         ...         ...   
301 2021-11-29  752.000000  755.000000  727.099976  739.349976  739.349976   
302 2021-11-30  740.000000  750.200012  722.650024  728.250000  728.250000   
303 2021-12-01  733.000000  737.400024  718.250000  722.250000  722.250000   
304 2021-12-02  723.900024  735.700012  714.150024  732.549988  732.549988   
305 2021-12-03  730.000000  734.650024  716.299988  718.349976  718.349976   

       Volume     Topline  Bottomline  Moving_Average_200  Movi

...

          Date        Open        High         Low       Close   Adj Close  \
0   2020-09-10  729.000000  735.900024  719.250000  726.799988  722.812256   
1   2020-09-11  725.799988  731.500000  720.250000  725.950012  721.966980   
2   2020-09-14  729.000000  730.250000  717.750000  721.599976  717.640808   
3   2020-09-15  723.000000  754.000000  722.650024  742.450012  738.376465   
4   2020-09-16  743.950012  756.500000  732.000000  752.599976  748.470703   
..         ...         ...         ...         ...         ...         ...   
301 2021-11-29  974.000000  998.000000  956.000000  965.000000  965.000000   
302 2021-11-30  965.000000  984.000000  958.599976  971.299988  971.299988   
303 2021-12-01  980.750000  986.700012  921.849976  928.150024  928.150024   
304 2021-12-02  937.950012  937.950012  917.500000  921.250000  921.250000   
305 2021-12-03  930.000000  930.000000  908.299988  912.049988  912.049988   

       Volume     Topline  Bottomline  Moving_Average_200  Movi

          Date         Open         High          Low        Close  \
0   2020-09-10  3765.500000  3765.500000  3700.050049  3719.250000   
1   2020-09-11  3724.000000  3828.000000  3720.000000  3772.250000   
2   2020-09-14  3788.899902  3796.000000  3711.000000  3719.500000   
3   2020-09-15  3720.050049  3766.500000  3715.000000  3734.100098   
4   2020-09-16  3742.000000  3860.000000  3737.149902  3844.500000   
..         ...          ...          ...          ...          ...   
301 2021-11-29  3550.000000  3550.000000  3469.100098  3528.600098   
302 2021-11-30  3514.000000  3565.000000  3501.100098  3545.500000   
303 2021-12-01  3550.500000  3606.699951  3528.000000  3535.250000   
304 2021-12-02  3540.000000  3607.300049  3530.000000  3578.500000   
305 2021-12-03  3557.000000  3593.449951  3550.000000  3553.750000   

       Adj Close  Volume      Topline   Bottomline  Moving_Average_200  \
0    3645.169434  357902  3765.500000  3719.250000                 NaN   
1    3697.1

          Date          Open          High           Low         Close  \
0   2020-09-10  19500.000000  19750.000000  19319.000000  19643.349609   
1   2020-09-11  19735.500000  19735.500000  19365.099609  19502.500000   
2   2020-09-14  19512.300781  19985.000000  19500.000000  19789.500000   
3   2020-09-15  19779.000000  20147.900391  19600.000000  20065.250000   
4   2020-09-16  20065.250000  20492.550781  19946.150391  20387.849609   
..         ...           ...           ...           ...           ...   
301 2021-11-29  25900.000000  25925.000000  25198.250000  25566.099609   
302 2021-11-30  25480.000000  26350.000000  25330.949219  26102.250000   
303 2021-12-01  26350.000000  26426.349609  25850.000000  26092.000000   
304 2021-12-02  26092.000000  26749.000000  26020.150391  26289.800781   
305 2021-12-03  26280.550781  26419.900391  25563.650391  25914.050781   

        Adj Close  Volume       Topline    Bottomline  Moving_Average_200  \
0    19601.380859   44713  19643.3

          Date         Open         High          Low        Close  \
0   2020-09-10  2920.000000  2957.399902  2888.250000  2948.750000   
1   2020-09-11  2949.149902  3007.699951  2948.750000  2992.250000   
2   2020-09-14  3014.949951  3039.000000  2956.000000  3028.500000   
3   2020-09-15  3028.500000  3051.699951  3005.050049  3037.149902   
4   2020-09-16  3033.000000  3105.000000  3024.649902  3056.199951   
..         ...          ...          ...          ...          ...   
301 2021-11-29  2525.000000  2528.949951  2446.149902  2490.800049   
302 2021-11-30  2476.000000  2512.649902  2424.050049  2449.300049   
303 2021-12-01  2480.000000  2480.000000  2437.050049  2448.199951   
304 2021-12-02  2428.000000  2483.199951  2408.899902  2472.699951   
305 2021-12-03  2485.000000  2497.449951  2451.050049  2462.449951   

       Adj Close   Volume      Topline   Bottomline  Moving_Average_200  \
0    2934.184570  1629594  2948.750000  2920.000000                 NaN   
1    2977

          Date       Open       High        Low      Close  Adj Close  Volume  \
0   2020-09-09  73.786797  74.050003  73.082497  73.786797  73.786797       0   
1   2020-09-10  73.550003  73.550003  73.012001  73.550003  73.550003       0   
2   2020-09-11  73.750000  73.750000  73.363998  73.527298  73.527298       0   
3   2020-09-14  73.478699  73.702400  73.244003  73.478600  73.478600       0   
4   2020-09-15  73.384003  73.723000  73.288498  73.614998  73.614998       0   
..         ...        ...        ...        ...        ...        ...     ...   
318 2021-11-29  75.052803  75.156502  74.535500  74.981598  74.981598       0   
319 2021-11-30  75.055000  75.206497  74.843002  75.054901  75.054901       0   
320 2021-12-01  75.056198  75.079300  74.719002  75.050903  75.050903       0   
321 2021-12-02  75.017899  75.259804  74.867798  75.017899  75.017899       0   
322 2021-12-03  74.980904  75.328400  74.929001  74.983498  74.983498       0   

       Topline  Bottomline 

In [107]:
from flask import Flask, render_template, jsonify, request

app = Flask(__name__)
cols = ['Date','NAME', 'Support_Level', 'Resistance_Level','Action','Close','Target','Stoploss','Expected_Within','Risk_to_Reward']

@app.route('/home')
def home():
    return render_template('index_template.html', RECOM_TABLE='', HEADER= render_template('header_home.html'))

@app.route('/recommendations')
def recommendations():
    global df
    dfs = df[df.Action.notnull()][cols]
    for i in ['Support_Level','Target','Stoploss','Resistance_Level','Expected_Within','Close']:
        dfs[i] = dfs[i].astype(int)
    s = dfs.style.format().hide_index()
    s = s.set_table_attributes('class="w3-table w3-hoverable"').render()
    return render_template('index_template.html', RECOM_TABLE=s, 
                           HEADER= render_template('header_recommendations.html'))

@app.route('/backtest')
def backtest():
    t = request.args.get('Ticker')
    d = request.args.get('Date')
    dfs = get_for_bt(d,t)
#     print(dfs)
    dfs = dfs[cols]
#     for i in ['Support_Level','Target','Stoploss','Resistance_Level','Expected_Within','Close']:
#         dfs[i] = dfs[i].astype(int)
    s = dfs.style.format().hide_index()
    s = s.set_table_attributes('class="w3-table w3-hoverable"').render()
    return render_template('index_template.html', RECOM_TABLE=s, HEADER= render_template('header_backtest.html'))

app.run(host='0.0.0.0', debug=True, use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://172.20.73.72:5000/ (Press CTRL+C to quit)
172.20.64.1 - - [04/Dec/2021 21:03:57] "GET /home HTTP/1.1" 200 -
172.20.64.1 - - [04/Dec/2021 21:03:57] "GET /static/logo_1.png HTTP/1.1" 304 -
172.20.64.1 - - [04/Dec/2021 21:03:59] "GET /recommendations HTTP/1.1" 200 -
172.20.64.1 - - [04/Dec/2021 21:03:59] "GET /static/logo_1.png HTTP/1.1" 304 -
